## Pandas

Author: Yujiang Peng

Date: Dec 20, 2019

1. 数据结构
2. 缺失值
3. 基本函数
4. 数据聚合与群运算
5. Data Wrangling: 合并、重塑、清洗、转化
6. 获取数据
7. 描述性统计方法
8. 时间序列

In [4]:
import pandas as pd

# 1. 数据结构
- s1、s2为序列series实例名称
- dict1为字典实例名称
## 1.1 series (一维)

In [ ]:
s1 = pd.Series([1,2], index = ['a','b']) #创建序列
s1 = pd.Series(dict1) #可将序列视为固定长度的字典；可替代字典
s1.values #获取序列值
s1['a'] #通过索引获取序列值
    s1[['b','a']] #通过索引获取序列值
s1.index #获取序列的索引
s1.name #获取名称属性；默认值为空None
    s1.index.name
s1 + s2 #具有相同索引的元素相加
s1.unique() #返回唯一值，但未排序

## 1.2 dataframe (二维)
- dataframe(DF)数据结构可视为序列的字典

In [ ]:
# 创建DF（由长度相同的列表或者Numpy数组创建）
dict1 = {'state':['Ohio','CA'], 'year':[2000,2001]}
df1 = pd.DataFrame(dict1) #列按顺序放置
df1 = pd.DataFrame(dict1, index=['row1','row2']) #设置(行)索引值
df1 = pd.DataFrame(dict1, columns=['year','state']) #列按给定顺序放置
# 创建DF（由嵌套字典创建，内部键值为行索引值）
dict1 = {'col1':  {'row1': 1, 'row2': 2}, 'col2': {'row1': 3, 'row2': 4} }
df1 = pd.DataFrame(dict1)

df1.columns # 获取列、行名
df1.index

df1.columns.name #获取名称属性（默认值为空None）
df1.index.name

df1.values #获取值；返回数据为二维数组形式

df1.['state'] #将列获取为序列
df1.state

df1.ix['row2'] #将行获取为序列
df1.ix[1]

df1['eastern'] = df1.state == 'Ohio' #创建新列

df1['eastern'] #删除列

df1.T #交换行和列；转置

## 1.3 panel date (三维)

In [ ]:
# 创建panel date（每一项均为DF）
import pandas_datareader.data as web
panel1 = pd.Panel({stk : web.get_data_yahoo(stk, '1/1/2000', '1/1/2010')    
for stk in ['AAPL', 'IBM']}) #数据维数2 (item) * 861 (major) * 6 (minor)

# 堆叠Df表
panel1 = panel1.swapaxes('item', 'minor')
panel1.ix[:, '6/1/2003', :].to_frame() #DF有to_panel()方法；与to_frame()相反
=> Stacked DF (with hierarchical indexing **) :
#  Open High Low Close Volume Adj-Close
#  major         minor
#  2003-06-01    AAPL
#                IBM
#  2003-06-02    AAPL
#                IBM

## 1.4 索引对象
即保存有axis label和其它元数据的不可变对象（如轴名）：
- 如Index, MultiIndex, DatetimeIndex, PeriodIndex
- 在创建序列或DF时从内部转换为索引的标签
- 可作为固定大小的集合，且类似于数组

## 1.5 分层索引
- 一个轴上的多层索引：用低维的形式处理高维数据

In [ ]:
# 多索引
series1 = Series(np.random.randn(6), index = [['a', 'a', 'a', 'b', 'b',\
         'b'], [1, 2, 3, 1, 2, 3]])
series1.index.names = ['key1', 'key2']

# 序列部分索引
series1['b']  # Outer Level
series1[:, 2] # Inner Level

# DF部分索引
df1['outerCol3','InnerCol2']               
    #or            
df1['outerCol3']['InnerCol2']

- 交换和排序层级

In [ ]:
# 交换层级（行的顺序不变，只交换两层）
swapSeries1 = series1.
swaplevel('key1', 'key2')

# 排序层级
series1.sortlevel(1) #根据内部第一层排序

# 交换和排序
# 最外层索引经排序后数据选择性能更好；调用sortlevel(0) or sort_index()
series1.swaplevel(0, 1).sortlevel(0) #行的顺序变化

- 按层级汇总统计信息
    - 序列和DF的多数函数均有level选项，可指定axis的层级

In [ ]:
df1.sum(level = 'key2') #具有相同层级key2的行求和
df1.sum(level = 'col3', axis=1) #列求和
#实际上利用了groupby

- 将dataframe的列作为索引

In [ ]:
# 新的采用列作为索引的DF
df2 = df1.set_index(['col3', 'col4']) #col3成为最外层索引；col4成为内层索引
    #col3, col4的值成为索引值；默认将会从DF数据中移除，可设置drop=False不移除
# reset_index与set_index相反，可见多层索引移变成列

# 2. 缺失值

In [ ]:
#Python
NaN - np.nan(not a number)
#Pandas
NaN或python内置的None表示缺失值
#采用pd.isnull(),  pd.notnull()或series1/df1.isnull()来检测缺失数据

## 2.1 筛选出缺失值

In [ ]:
# dropna()仅返回非空数据，不修改源数据
df1.dropna() #去掉所有含缺失值的行
df1.dropna(axis=1) #去掉所有含缺失值的列
df1.dropna(how = 'all') #去掉数据全部缺失的行
df1.dropna(thresh = 3) #去掉数据数量少于3的行

## 2.2 填充缺失数据

In [ ]:
df2 = df1.fillna(0) #将所有缺失值填充为0
df1.fillna('inplace = True') #在数据内修改
df1.fillna({'col1' : 0, 'col2' : -1}) #为每一列采用不同的填充方法
df1.fillna(method = 'ffill', limit = 2) #仅填充前两个缺失值

# 3. 基本函数
## 3.1 索引（切片/分割子集）

In [ ]:
# 1. 终点包含在pandas标签切片中，如series1['a':'c']；而python切片则不包含
# 2. 注意pandas非标签（如整数）切片也不包含
#列索引
df1['col1']
df1[ ['col1', 'col3'] ]
#行索引
df1.ix['row1']
df1.ix[ ['row1', 'row3'] ]
#同时由列和行索引
df1.ix[['row2', 'row1'], 'col3']
#布尔值索引
df1[ [True, False] ]
df1[df1['col2'] > 6] #返回col2值大于6的df
    # df1['col2'] > 6返回一个布尔值序列，决定各值是否出现在结果中。
    # 避免使用整数索引，它可能对引起故障（如series1[-1]）；如果一定要使用基于位置
    # 的索引；那么序列使用iget_value()，DF则使用irow/icol()

## 3.2 删除行/列

In [ ]:
# 删除操作返回一个新的对象（如DF)
#删除行（默认值为aixs=0）
df1.drop('row1')
df1.drop(['row1', 'row2'])
#删除列
df1.drop('col2', axis = 1)

## 3.3 重新索引

In [ ]:
#修改索引值
date_index = pd.date_range('01/23/2010', periods = 10, freq = 'D')
#重新索引，默认是修改行索引
df1.reindex(date_index)
#用0代替缺失索引值
df1.reindex(date_index, fill_value = 0) 
#重新索引列
df1.reindex(columns = ['a', 'b'])
#重新索引行和列
df1.reindex(index = [..], columns = [..])
#简洁索引
df1.ix[[..], [..]]

## 3.4 算术运算与数据对齐

In [ ]:
#执行运算 df1 + df2 时，对于不重叠的部分索引，进行内部数据对齐后索引值会变成NaN
# 1. 使用0来代替不重叠的部分索引，而不是NaN
df1.add(df2, fill_value = 0)
# 2. 实用操作
df1 - df1.ix[0] #每一行均减去df1的第一行

## 3.5 排序(sorting)与排名(ranking)

- 索引/列排序 sort index/column

In [ ]:
# 1. sort_index()函数返回一个新的、排序后的对象。默认升序 ascending = True
# 2. 默认行排序，将参数 axis=1 可进行列排序
## 索引/列排序意味着排序行/列标签，而不是排序数据

- 数据排序 sort data

In [ ]:
# 缺失值 np.nan 默认排在序列 Series 的末尾
#序列排序
sortedS1 = series1.order()
series1.sort() # In-place sort  
#DF排序
 df1.sort_index(by = ['col2', 'col1'])  #先col2，再col1

- 排名 ramking

In [ ]:
#多个排名相同则采用其排名的平均值
series1.rank() #返回各元素排名
df1.rank(axis = 1) #排名各行的值

## 3.6 函数应用

- Numpy可以很好地处理pandas对象：np.abs(df1)

In [ ]:
# 用函数处理各列或者各行（默认处理各行，axis=0）
f = lambda x: x.max() - x.min() # 返回一个标量值
#
# 下面两个函数均返回多个值
def f(x): return  # 对 x 应用函数 f
Series([x.max(), x.min()]) # 返回最大最小值
#
df1.apply(f)

# 用函数处理各个元素
f = lambda x: '%.2f' %x # 均保留两位小数
df1.applymap(f)  

## 3.7 唯一性、个数

In [ ]:
series1/df1.index.is_unique # 可检查唯一性
pd.value_counts() # 数值数量

# 4. 数据聚合与群组运算
- 分类数据集且应用函数到各组，不管是聚集还是转换
- 注意：时间序列（Time Series）数据参见时间序列部分；resampling为groupby的特殊用例

## 4.1 groupby 

In [ ]:
# 计算组的平均值
df1.groupby('col2').mean()
# groupby多个键
df1.groupby([df1['col2'], df1['col3']]).mean()
# groupby对象：仅计算内部数据关于组键df1['col2']
grouped = df1['col1'].groupby(df1['col2'])
grouped.mean() # gets the mean of each group formed by 'col2'

# 索引groupby对象
# 选取col1来聚集
df1.groupby('col2')['col1'] 
    # or
df1['col1'].groupby(df1['col2'])
# 注意：缺失值均从结果中排除。

# 1. 通过groupby对象迭代
# 生成元组序列，元组每个元素含有两个数据（组名和数据）
# 组名为单值；数据为通过组名筛选后的DF
for name, groupdata in df1.groupby('col2'): 
# 如果groupby多个键，那么元组第一个元素为元组的键值
for (k1, k2), groupdata in df1.groupby(['col2', 'col3']):
#
# 将组转换为字典
dict(list(df1.groupby('col2'))) # col2的值将成为字典的键
# 通过dtype来group列
grouped = df1.groupby([df1.dtypes, axis = 1)
dict(list(grouped))

# 2. 通过函数分组
# 每个被传递作为群组键的函数都会被每个值（默认为行索引）调用一次。
# 且返回值被作为组名（假设含索引已被命名）。
df1.groupby(len).sum() # 返回一个行索引为名字长度的DF；因此，具有相同名字长度的名字将被求和。列名不变。

## 4.2 数据聚合
- 表示从数组产生标量值的数据转换，如平均值、最大值等

In [ ]:
# 自定义函数
def func1(array): ...
grouped.agg(func1)
# 使用列名作为函数名来获取DF
grouped.agg([mean, std])
# 使用自定义的列名来获取DF
grouped.agg([('col1', mean), ('col2', std)])
# 对不同列采用不用函数
grouped.agg({'col1' : [min, max], 'col3' : sum})

## 4.3 群组操作与转换
- **Agg()**是数据转换的特例，**one-dimensional array to scalar**的简称
- **Transform()**是一种特殊的数据转换：
    - 它将一个函数应用到每个组，如果产生的是标量值，这个值将被置于该组的**每行**
    - 被传递的函数要么产生一个标量值，要么产生一个具有相同大小的转换后的数组
- 通用目的转换**apply()**

In [ ]:
# you_func仅需返回一个pandas对象或者一个标量
df1.groupby('col2').apply(your_func1)

# 例一：年度与SPX的相关性
# “close_price” is DF with stocks and SPX closed price columns and dates index 
returns = close_price.pct_change().dropna()
by_year = returns.groupby(lambda x : x.year) 
spx_corr = lambda x : x.corrwith(x['SPX'])
by_year.apply(spx_corr)

# 例二：探索回归
import statsmodels.api as sm
def regress(data, y, x):
   Y = data[y]; X = data[x]
   X['intercept'] = 1
   result = sm.OLS(Y, X).fit()
   return result.params    
by_year.apply(regress, 'AAPL', ['SPX'])

# 5. Data Wrangling: 合并、重塑、清洗、转化

## 5.1 组合与合并数据

In [ ]:
1. pd.merge() aka database 'join' # 在DF中通过一个或多个键连接行
##列合并（常用）
df3 = pd.merge(df1, df2, on = 'col2')
        # 使用所有重叠的列名作为键来合并。好的做法是列出这些键。on = ['col2', 'col3']
        # 如果df1和df2中键名不同，可使用选项：left on = 'lkey', right on = 'rkey'
    # INNER为默认值，也可有其他选择: how = 'outer/ left/ right'
    # 在df3中df1和df2的索引被丢弃
##行合并
df3 = pd.merge(df1, df2, left_index = True, right_index = True)
    # 将索引作为合并键：具有相同索引值的aka行被合并。


2. pd.concat() # 沿某个轴堆叠（默认沿着行：axis = 0）
df3 = pd.concat([df1, df2], ignore_index = True) # 丢弃df3中索引
    # 如果df1有2行，df2有3行，那么合并后df3有5行。


3. combine_first() # 合并具有重叠的数据，补充缺失值。
df3 = df1。combine_first(df2)
    # df1和df2的索引全部或部分重叠。
    # 如果某行存在于df1而不存在与df2中，这行将出现在df3中。
    # 如果df1和df2中分别有一行具有相同的索引值，但是df1中行的某列值为空，则在df3中改行将从df2中行获得数据。

## 5.2 重塑和旋转数据

In [ ]:
1. 重塑具有多层索引的数据
series1 = df1.stack()
    # 旋转（最内层*）列至行作为最内层的索引，使得series具有多层索引。
df1 = series1.unstack()
    # 旋转（最内层*）行至列作为最内层列。
        # 默认为stack/unstack最内层。如果想要不同的层，如stack(level = 0)则为最外层。
# 注意：unstacking可能会导致丢失数据，如果层中值有缺失。stacking会默认过滤掉缺失值，如data.unstack() .stack()


2. 旋转数据
# 在数据库和CSV中常用的保存多个时间序列的格式为
Long/Stacked Format: 'date, stock_name, price'
# 然而，一个具有上述三个列数据的DF很难处理。因此，更推荐'wide'格式：'date'作为行索引，'stock_name'作为列，'price'作为DF数据值。
pivotedDF2 = df1.pivot('date', 'stock_name', 'price')
    # 例如 pivotedDF2：
    #             AAPL     IBM     JD
    # 2003-06-01  120.2    110.1   20，8

## 5.3 常用操作

In [ ]:
1. 移除重复行
serial1 = df1。duplicated() # 布尔值序列，表示每行是否重复。
df2 = df1.drop_duplicates() # 重复的行被移除


2. 基于列值添加新列
df1['newCol'] = df1['col2'].map(dict1) # 将col2的值映射为dict1的键，得到dict1的值。
df1['newCol'] = df1['col2'].map(func1) # 将一个函数应用到col2的每个值。


3. 替换值
# 替换不是In-Place, 
df2 = df1.replace(np.nan, 100)
# 同时替换多个值
df2 = df1.replace([-1, np.nan], 100)
df2 = df1.replace([-1, np.nan], [1, 2])
# 变量也作为字典
df2 = df1.replace({-1: 1, np.nan : 2})


4. 重命名轴索引
## 将索引转换为大写
df1.index = df1.index.map(str.upper)
## 将'row1'重命名为'newRow1'
df2 = df1.rename(index = {'row1': 'newRow1'}, columns = str.upper) # 可选项，inplace = True


5. 离散化和binning
## 连续数据通常被离散化为bins来分析
    # 将数字(18-26], (26-35]分成两个Bins
    bins = [18, 26, 36]
    cat = pd.cut(array1, bins, labels=[..]) # cat是类别对象
    pd.value_counts(cat)
    cat = pd.cut(array1, numofBins) # 基于array1的最小最大值计算等长的bins
    cat = pd.qcot(array1, numofBins) # 基于样本数量划分数据，大小大致相同的bins


6. 检测和过滤离群值outliers
## any()沿着轴检查各元素是否为真。默认沿着列（axis = 0）检测。
df1[(np.abs(df1) > 3).any(axis = 1)] # 选择含有值的绝对值大于三的所有行。

# 另一个有用的函数：np.sign()  返回1或者-1


7. 排列组合与随机取样 permutation and ramdom smapling
randomOrder = np.random.permutation(df1.shape[0])
df2 = df1.take(randomOrder)


8. 计算指示符/虚拟变量 indicator/dummy variables
## 如果DF中某列有K个不同的值，得到一个含有K列0和1的指示符DF。1表示存在，0表示不存在。
dummyDF = pd.get_dummies(df1['col2'], prefix = 'col-') # 将前缀添加到K列名

# 6. 获取数据

## 6.1 文本格式（CSV）

In [ ]:
df1 = pd.read_csv(file/URL/file-like-object, sep = ',', header = None)
    # 类型推断：无需表明哪些列为数值型、整型、布尔型，或字符串
    # 在pandas中，源数据中的缺失数据通常为空字符串、NA、-1、#IND，或NULL。可通过下列选项指出缺失值：na_values = ['NULL']
    # 默认分隔符为逗号comma
    # 默认第一行为列头

df1 = pd.read_csv(.., names = [..])
    # 明确指出列头，也表明第一行为数据

df1 = pd.read_csv(.., names = [.., 'date'], index_col = 'date')
    # 想要使'date'列成为返回DF的行索引

df1.to_csv(filepath/sys.stdout, sep = ',')
    # 缺失值在输出中呈现为空字符串。因此，最好添加选项：na_rep = 'NULL'
    # 默认写行和列的标签。可通过选项禁用：index = False, header = False

## 6.2 JSON (javascript object notation) 数据

In [ ]:
## 在浏览器和其他应用程序间通过HTTP请求传递数据的标准格式之一
## 比表格化文本CSV更灵活的数据格式

# 将JSON字符串转化为Python表格
data = json.load(jsonObj)

# 将Python对象转化为JSON
asJson = json.dumps(data)

# 从JSON创建DF
df1 = pd.DataFrame(data['name'], columns = ['filed'])

## 6.3 XML和HTML数据

In [ ]:
## HTML:
doc = lxml.html.parse(urlopen('http://..')).getroot()
tables = doc.findall('.//table')
rows = tables[1].findall('.//tr')

## XML：
lxml.objectify.parse(open(filepath)).getroot()

# 7. 描述性统计方法

In [ ]:
### 相较于同样方法的ndArray, pandas中的描述性统计方法自底向上排除缺失数据
### NA（如NaN）值被排除。可通过选项禁用：skipna = False

# 列之和（使用axis = 1来求各行之和）
serial1 = df1.sum()

# 返回获得的最小/最大值索引标签
df1.idxmin()
df1.idxmax()

# 简要统计（如个数、均值、标准差等）
# 在非数值数据中，部分统计量（如个数、唯一性）
df1.describe()

## 7.1 相关性及协方差 correlation and covariance

In [ ]:
cov(), corr()
corrwith() # 成对相关性：aka计算一个DF与一个序列。若输入不是序列，而是另一个DF,则
            #将计算相匹配的列名的相关性，如returns.corrwith(volumes)

# 实例：相关性
import pandas_datareader.data as web
data = {}
for ticker in ['AAPL', 'JD']:
    data[ticker] = web.get_data_yahoo(ticker, '1/1/200', '1/1/2010')
    prices  = pd.DataFrame({ticker: d['AdjClose'] for ticker, d in data.iteritems()})
    volumes = ...
returns = prices.pct_change()
returns.AAPL.corr(returns.JD)
    # 序列corr()计算两个序列中重叠、non-NA、且通过索引对齐的值的相关性。

# 8. 时间序列

In [11]:
## Python表示日期和时间的标准库数据类型
#import datetime time calender
    # datetime应用广泛，它同时保存日期和精确到微秒的时间
## Pandas表示日期和时间的数据类型
#import Timestamp  # 可代替datetime对象


# 将字符串转换为DataTime
from datetime import datetime
datetime.strptime('8/8/2008', '%m/%d/%Y')

# 获取当前时间
now = datetime.now()

# DataTime计算
from datetime import timedelta
datetime(2011, 1, 8) + timedelta(12) => 2011-01-20
    # timedelta表示两个datetime对象的时间差

# 将字符串转化为Pandas Timest类型
timestamps = pd.to_datetime(['8/8/2008', ..])
    # NaT (Not a Time)在Pandas中表示Timestamp数据的NA值
pd.to_datetime('') => NaT
pd.isnull(NaT) => True # 缺失值，如：空字符串

datetime.datetime(2011, 1, 20, 0, 0)

## 8.1 Panda时间序列

In [ ]:
# 创建时间序列
ts1 = pd.Series(np.random.randn(2), index = [ datetime(2011, 1, 2), ... ])
ts1 = pd.Series(..., index = pd.date_range('1/1/2000', periods = 1000))
    # 每隔一天，一共1000天
    # ts1.index为DatetimeIndex Panda类
    # 索引值ts1.index[0]是Panda Timestamp对象，其使用NumPy的datetime64类型储存纳秒精度的时间戳
    # 时间戳类保存有频率信息和时区
    # ts1.index.dtype => datetime64[ns]

### 索引（切片/子集）
    变量可为一个string date、datetime、或Timestamp

In [ ]:
# 选取2001年
ts1['2001']
df1.ix['2001']

# 选取2001 6月
ts1['2001-06']

# 选取时间段
ts1['1/1/2001':'8/1/2001']

# 选取从某天开始的时间
ts1[datetime(2001, 1, 8):]

### 常用操作

In [ ]:
# 获取某天之前的时间序列
ts1.truncate(after = '1/8/2011')

## 8.2 日期范围、频率、shifting

In [ ]:
# Pandas里的通用时间序列无规律，aka没有固定的频率。
# 但是通常需要固定的频率，如每日、每月等

# Resampling部分
## 转换为固定的每日频率
## 如需要，引入缺失值NaN
ts1.resample('D', how = ...)


1. 频率及日期偏移
# Pandas中频率由基础频率和乘法器组成。基础频率通常指一个字符串别名，如'M', 'H'
freq = '4H'
freq = '1h30min'
    # 标准美国标准股每月满期为每月第三个周五：freq = 'WOM-3FRI'

2. 生成日期范围
# 默认频率是每日
pd.date_range(begin, end)
pd.date_range(begin or end, periods=n)
    # 选项freq = 'BM'表示每月最后一个工作日

3. 日期移位
## 移位表示将数据沿着时间前移或后移
## 序列和DF的 shift() 进行简单移位，aka索引不移位，只移值。

# ts1为每日数据
ts1.shift(1) # 将昨天的值移到今天，今天的移到明天，等

# ts1为任意时间序列数据。数据移位3天。
ts1.shift(3, freq = 'D')
ts1.shift(1, freq = '3D')

# 常用移位操作：计算%变化
ts1 / ts.shift(1) -1

    # shift()的返回值中，部分数据值可能为NaN

## 其他移位方法
from pandas.tseries.offsets import Day, MonthEnd
datetime(2008, 8, 8) + 3*Day() => 2008-08-11
datetime(2008, 8, 8) + MonthEnd(2) => 2008-09-30
MonthEnd().rollforward(datetime(2008, 8, 8)) => 2008-08-31

## 8.3 时区操作

In [ ]:
## Daylight saving time (DST)复杂
## UTC为当前国际通用标准。时区表示为从UTC的偏移

1. Python时区（从第三方库pytz）
# 获取时区名
pytz.common_timezones
# 获取时区对象
pytz.timezone('US/Eastern')

2. 本地化与转换
# 默认的时间序列是简单时区
ts1.index.tx => NotImplemented
# 创建时间序列时表明时区
pd.date_range(tz = 'UTC')
# 从简单时区本地化
ts1_utc = ts1.tz_localize('UTC')
# 更换时区
ts1_eastern = ts1_utc.tz_convert('US/Eastern')

3. 时区感知时间戳对象
    # 若需合并两个不同时区的时间序列，如ts1+ts2，时间戳将自动对齐，结果为UTC。
stamp_utc = pd.Timestamp('2008-08-08 03:00', tz = 'UTC')
stamp_eastern = stamp_utc.tz_convert(...)

# Pandas时间计算  daylight savings time
stamp = pd.Timestamp('2012-11-04 00:30', tz = 'US/Eastern') => 2012-11-04-00:30:00 -400 EDT
stamp + 2 * Hour() => 2012-11-04-01:30:00 -400 EST

## 8.4 重采样

In [ ]:
## 将时间序列从一个频率转换到另一个频率的过程
1. 降采样  吸收高频数据为低频数据
    ts1.resample('M', how = 'mean') => index: 2000-01-31, 2000-02-29, ...

    ts1.resample('M', ..., kind = 'period') => index: 2000-01, 2000-02,...
        # 'period'-使用时间区间
    
    # ts1是从1到100的一分钟数据
    00:00:00, 00:01:00, ...
    ts1.resample('5min', how = 'sum') => 00:00:00 15 (aka: 1+2+3+4+5)
                                         00:05:00 40
    # 默认左侧包含
    # 选项 closed = 'right' 可切换到右侧包含。还可 label = 'right'
    00:00:00 1
    00:05:00 20 (aka: 2+3+4+5+6)

    ts1.resample('5min', how = 'ohlc') # 返回4列的DF: open, high, low, close

2. 升采用和插值  通过插值使低频变为高频。默认情况下引入缺失值（NaN）
        # 插值仅应用于行
    df1.resample('D', fill_method = 'ffill')
        # Forward fills values，如，索引为3的缺失值将复制索引为2的值。

## 8.5 时间序列绘图

In [ ]:
# 实例：源数据格式-第一列为日期
# 使用第一列作为索引，然后解析索引值作为日期
prices = pd.read_csv(.., parse_date = True, index_col = 0)
px = prices[['AAPL', 'IBM']]
px = px.resample('B', fill_method = 'ffill')
px['AAPL'].plot()
px['AAPL'].ix['01-2008':'03-2012'].plot()
px.ix['2008'].plot()

## 8.6 移除窗口函数

In [ ]:
# 像其它统计函数一样，这些函数也自动排除缺失数据
pd.rolling_mean(px.AAPL, 200).plot()
pd.rolling_std(px.AAPL.pct_change(), 22, min_periods = 20).plot()
pd.rolling_corr(px.AAPL.pct_change(), px.IBM.pct_change(), 22).plot()

## 8.7 性能